# Cannon Game

필요한 library import

In [13]:
# Cannon
import time
import random
import math
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np

Joystick클래스 선언

<choi++> LCD boundary check

In [14]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # Create blank image for drawing.
        # Make sure to create image with mode 'RGB' for color.
        self.width = self.disp.width
        self.height = self.disp.height

    # choi++ check boundary
    ''' return.. True: in LCD boundary
                 False: out of LCD '''
    def inLCD(self, position):
        return (position[0] < self.width and position[1] < self.height) \
                 or (position[2] >= 0 and position[3] >= 0)

캐릭터 선언 (대각선 이동 가능)

joystick, Image, Draw 객체 생성

In [15]:
joystick = Joystick()
my_image = Image.new("RGB", (joystick.width, joystick.height)) #도화지!
my_draw = ImageDraw.Draw(my_image) #그리는 도구!

### 캐릭터 수정 및 캐릭터가 쏘는 총알 구현 및 동작 구현

In [16]:
class Character:
    def __init__(self, width, height):
        self.appearance = 'circle'
        self.state = None
        self.position = np.array([width/2 - 20, height/2 - 20, width/2 + 20, height/2 + 20])
        # 총알 발사를 위한 캐릭터 중앙 점 추가
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#FFFFFF"

    def move(self, command = None):
        if command['move'] == False:
            self.state = None
            self.outline = "#FFFFFF" #검정색상 코드!
        
        else:
            self.state = 'move'
            self.outline = "#FF0000" #빨강색상 코드!

            if command['up_pressed']:
                self.position[1] -= 5
                self.position[3] -= 5

            if command['down_pressed']:
                self.position[1] += 5
                self.position[3] += 5

            if command['left_pressed']:
                self.position[0] -= 5
                self.position[2] -= 5
                
            if command['right_pressed']:
                self.position[0] += 5
                self.position[2] += 5
                
        #center update
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2]) 

실행시켜보면 문제점이 무엇인가요?
- 가만히 있는 상태에서 발사버튼(A)를 누르면 bullet이 그려진 후, 움직이지 않음
- bullet을 담는 bullets 리스트에 bullet이 쌓이기만 할 뿐, 사라지지 않음 -> 메모리가 부족해지고 점점 동작 속도가 느려지는 것도 체감 가능  

어떻게 하면 고칠 수 있을지 생각해보고 고쳐보세요😀

In [17]:
class Cannon:
    def __init__(self, pos):
        self.appearance = 'circle'
        self.state = None
        self.c = pos
        self.pos = np.array([pos[0] - 10, pos[1] - 10, pos[0] + 10, pos[1] + 10])
        self.angle = 45
        self.speed = 70
        self.outline = "#FFFFFF"

    def set(self, command = None):
        if command == 'up':
            self.angle += 0.5
        if command == 'down':
            self.angle -= 0.5
        if command == 'right':
            self.speed += 1
        if command == 'left':
            self.speed -= 1

### 적 생성 및 HIT 액션 적용

enemy는 우리가 만든 Character와 굉장히 유사합니다.  
우선은 움직이지 않는 적을 만들어봅시다.

In [18]:
class Enemy:
    def __init__(self, spawn_position):
        self.appearance = 'circle'
        self.state = 'alive'
        self.position = np.array([spawn_position[0] - 10, spawn_position[1] - 10, spawn_position[0] + 10, spawn_position[1] + 10])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#00FF00"

포물선 bullet 구현 (s=vt, s=(1/2)gt^2, vx=cos(v0), vy=sin(v0))  
초기조건: (x0, y0)=시작점, v0=속도, rad=각도,  dt=interval, g=중력가속도(9.8)  
수평속도(vx): v0 * cos(rad), 수직속도(vy): v0 * sin(rad)  
bullet의 x 위치= x0 + vx * dt  
bullet의 y 위치= y0 + vy * dt - 0.5 * g * dt^2  

In [19]:
class Bullet:
    BR = 3  # radius of bullet
    BR2 = BR*2 
    G = 9.8 # gravitational constant
    def __init__(self, pos, angle, speed):
        self.appearance = 'circle'
        rad = math.radians(angle)
        self.vx = speed * math.cos(rad)
        self.vy = speed * math.sin(rad)
        self.damage = 10
        self.p = np.array([pos[0]-BR, pos[1]-BR, pos[0]+BR, pos[1]+BR])
        self.p0 = self.p
        self.c = pos
        self.c0 = self.c
        self.t = 0
        self.state = 'move'
        self.outline = "#0000FF"

    def move(self):
        dx = int(self.vx*self.t * 0.02)
        dy = int((self.vy*self.t - 0.5*G*self.t**2) * 0.02)
        self.p0 = self.p                            # save previous position
        self.p[0] += dx
        self.p[2] += dx
        self.p[1] -= dy
        self.p[3] -= dy
        self.c0 = self.c                            # save previous center
        self.c = np.array([(self.p[0] + self.p[2]) / 2, (self.p[1] + self.p[3]) / 2])
        self.t += 0.5

    def ground(self):
        ''' True if bullet is down on the ground '''
        return self.p[1] > 240
             
    def collision_check(self, enemys):
        for enemy in enemys:
            collision = False                               # choi++
            if enemy.state == 'alive':                      # choi++
                collision = self.overlap(enemy.position)
            if collision:
                enemy.state = 'die'
                self.state = 'hit'

    def overlap(self, other_p):
        ''' check if other_p is in the box of ego_p0 ~ ego_p
        return : True : if overlap
                False : if not overlap '''
        if (self.p[2]>other_p[0] and self.p[0]<other_p[2] and self.p[3]>other_p[1] and self.p[1]<other_p[3]):
            return True
        else:
            dx = (self.c[0] - self.c0[0]) / BR2
            dy = (self.c[1] - self.c0[1]) / BR2
            i = 0
            while (self.c0[0]+(dx*i) > self.c[0]):
                
            return False
        '''
        if (ego_p0[1] < ego_p[1]):
            return ego_p0[0] < other_p[2] and ego_p[2] > other_p[0] \
                and ego_p0[1] < other_p[3] and ego_p[3] > other_p[1]
        else:
            return ego_p0[0] < other_p[2] and ego_p[2] > other_p[0] \
                and ego_p0[3] > other_p[1] and ego_p[1] > other_p[3]
        if (ego_p0[1] < ego_p[1]):
            return ego_p0[0] < other_p[2] and ego_p[2] > other_p[0] \
                and ego_p0[1] < other_p[3] and ego_p[3] > other_p[1]
        else:
            return ego_p0[0] < other_p[2] and ego_p[2] > other_p[0] \
                and ego_p0[3] > other_p[1] and ego_p[1] > other_p[3]
        '''

<choi++>  
살아있는 enemy에 대해서만 collision check 한다.  
화면 밖으로 나가는 bullet은 list에서 제거하여 메모리와 CPU를 절약한다.  
죽은 enemy는 list에서 제거한다.  
정지해 있는 bullet은 제거한다. (bullet 생성시에 state 설정: 'move')

In [20]:
my_cannon = Cannon((20, 220))
# my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))

#rx = random.randrange(180, 240)
#ry = random.randrange(180, 240)
#enemy = Enemy((rx, ry))
enemys_list = []

bullet = None
while True:
    command = None
    
    if not joystick.button_U.value:     # up pressed
        command = 'up'
    if not joystick.button_D.value:     # down pressed
        command = 'down'
    if not joystick.button_L.value:     # left pressed
        command = 'left'
    if not joystick.button_R.value:     # right pressed
        command = 'right'
    if not joystick.button_A.value:     # A pressed
        bullet = Bullet(my_cannon.c, my_cannon.angle, my_cannon.speed)
    if not joystick.button_B.value:     # B pressed
        rx = random.randrange(120, 240)
        ry = random.randrange(120, 240)
        enemy = Enemy((rx, ry))
        enemys_list.append(enemy)

    my_cannon.set(command)
    if bullet != None:
        bullet.collision_check(enemys_list)
        bullet.move()
#        print(bullet.c[0], bullet.c[1])
                
    my_draw.rectangle((0, 0, joystick.width, joystick.height), fill = (255, 255, 255, 100))
    my_draw.ellipse(tuple(my_cannon.pos), outline = my_cannon.outline, fill = (0, 0, 0))

    dx = int(my_cannon.speed * math.cos(math.radians(my_cannon.angle))/2)
    dy = int(my_cannon.speed * math.sin(math.radians(my_cannon.angle))/2)
    my_draw.line((my_cannon.c[0], my_cannon.c[1], my_cannon.c[0]+dx, my_cannon.c[1]-dy), fill=(0,0,230), width=2)
    
    for enemy in enemys_list:
        if enemy.state == 'alive':          # choi++
            my_draw.ellipse(tuple(enemy.position), outline = enemy.outline, fill = (255, 0, 0))
        elif enemy.state == 'die':          # choi++
            enemys_list.remove(enemy)       # choi++

    #   for bullet in bullets:
    if bullet != None:
        if (bullet.state == 'move') and (not bullet.ground()):      # choi-+
            my_draw.ellipse(tuple(bullet.p), outline = bullet.outline, fill = (0, 0, 255))
        else:                               # choi++
        #    bullets.remove(bullet)          # choi++
            bullet = None
    

    #좌표는 동그라미의 왼쪽 위, 오른쪽 아래 점 (x1, y1, x2, y2)
    joystick.disp.image(my_image)

KeyboardInterrupt: 

총알을 맞추면 적들이 잘 사라지나요?  
아직 미흡한 점이 무엇인지 생각해보고 수정해봅시다!😀
- Enemy 역시 그냥 눈으로만 안보이는 척 하는 것일 뿐, list 내에 살아 있음. -> 메모리 관리 필요
- Enemy가 총알에 맞았을 시, 한 방에 죽는 것이 아닌 체력이 있다면?